In [1]:
import sys
sys.path.insert(0, '../../../fastshap_tf/')
from fastshap import ResizeMask

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
import pickle
import numpy as np
import shap
from tqdm.notebook import tqdm
import time

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds

from datetime import datetime
import os

In [4]:
from tensorflow.keras.layers import (Input, Layer, Dense, Lambda, 
                                     Dropout, Multiply, BatchNormalization, 
                                     Reshape, Concatenate, Conv2D, Permute)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [5]:
#Select GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

## Load Data

In [6]:
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'imagenette/full-size-v2',
    split=['train', 'validation[:50%]', 'validation[-50%:]'],
    as_supervised=False,
    with_info=True
)

### Batch Data

In [7]:
def batch_data(dataset, fn, batch_size=32):
    dataset = dataset.map(fn)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

### Reformat Data

In [8]:
BATCH_SIZE = 32

def reformat(input_dict):
    
    i = input_dict['image']
    i = tf.cast(i, tf.float32)
    i = tf.image.resize_with_crop_or_pad(i, 224, 224)
    i = tf.keras.applications.resnet50.preprocess_input(i)
    
    l = tf.one_hot(input_dict['label'], depth = 10)
    
    return (i, l)

ds_train = batch_data(ds_train, reformat, BATCH_SIZE)
ds_val = batch_data(ds_val, reformat, BATCH_SIZE)
ds_test = batch_data(ds_test, reformat, BATCH_SIZE)

## Load Model

In [9]:
from tensorflow.keras.applications.resnet50 import ResNet50

INPUT_SHAPE = (224,224,3)

base_model = ResNet50(
    include_top=True, weights='imagenet', 
    input_shape=INPUT_SHAPE
)
base_model.trainable = False

model_input = Input(shape=INPUT_SHAPE, dtype='float32', name='input')

net = base_model(model_input)
out = Dense(10, activation='softmax')(net)

model = Model(model_input, out)

model_weights_path = 'model/20210511_21_28_36/model_weights.h5'

model.load_weights(model_weights_path)
model.trainable = False

# Train Evaluator

### Prepare Dataset w/ Model Predictions

In [10]:
@tf.function
def make_prediction_data(x, y):
    with tf.device("gpu:3"):
        y_model = model(x)
    
    return (x, y_model)

with tf.device("gpu:3"):
    ds_train_pred = ds_train.map(make_prediction_data)
    ds_val_pred = ds_val.map(make_prediction_data)
    ds_test_pred = ds_test.map(make_prediction_data)

### Save Dir

In [11]:
save_dir = 'evaluator'
model_dir = os.path.join(os.getcwd(), save_dir)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

### Initialize Model

In [12]:
class Random_Bernoulli_Sampler(Layer):
    '''
    Layer to Sample r
    '''
    def __init__(self, num_features, **kwargs):
        super(Random_Bernoulli_Sampler, self).__init__(**kwargs)
        
        self.num_features = num_features
        
        
    def call(self,  logits):
        batch_size = tf.shape(logits)[0]
        
        u = tf.random.uniform(shape=(batch_size, self.num_features),
                                    minval=np.finfo(
                                        tf.float32.as_numpy_dtype).tiny,
                                    maxval=1.0) 

        r = K.cast(tf.stop_gradient(u > 0.5), tf.float32)
        
        return r

In [13]:
base_model = ResNet50(
    include_top=False, weights='imagenet', 
    input_shape=INPUT_SHAPE, pooling='avg'
)
base_model.trainable = True

model_input = Input(shape=INPUT_SHAPE, dtype='float32', name='input')
R = Random_Bernoulli_Sampler(14*14)(model_input)
R = ResizeMask(in_shape=INPUT_SHAPE, mask_size=14*14)(R)
xr = Multiply()([model_input, R])

net = base_model(xr)
out = Dense(10, activation='softmax')(net)

evaluator_model = Model(model_input, out)

### Train Model

In [14]:
LR = 1e-3

# Metrics
METRICS = [ 
  tf.keras.metrics.AUC(name='auroc'),
  tf.keras.metrics.AUC(curve='PR', name='auprc'),
  tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='accuracy'),
]

# Model Checkpointing
model_weights_path = os.path.join(model_dir, 'model_weights.h5')
checkpoint = ModelCheckpoint(model_weights_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only=True)

# LR Schedule
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.95, patience=5, 
                             verbose=1, mode='min', cooldown=1, min_lr=1e-3/10)

# Early Stopping 
earlyStop = EarlyStopping(monitor="val_loss", mode="min", patience=20) 

# Compile Model
CALLBACKS = [checkpoint, earlyStop, reduceLR]
OPTIMIZER = tf.keras.optimizers.Adam(LR)

evaluator_model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=METRICS,
)

# Train Model
evaluator_model.fit(ds_train,
          epochs = 100,
          validation_data = ds_val,
          callbacks = CALLBACKS)

# Get Checkpointed Model
print(model_weights_path)
evaluator_model.load_weights(model_weights_path)
evaluator_model = Sequential(   
            [l for l in evaluator_model.layers[-2:]]
        )
evaluator_model.trainable = False

Epoch 1/100
296/296 [==============================] - ETA: 0s - loss: 1.2913 - auroc: 0.9084 - auprc: 0.6571 - accuracy: 0.6109
Epoch 00001: val_loss improved from inf to 1.39231, saving model to /gpfs/data/paulab/nj594/fast_shap/experiments/images/imagenette/evaluation/evaluator/model_weights.h5
296/296 [==============================] - 152s 515ms/step - loss: 1.2913 - auroc: 0.9084 - auprc: 0.6571 - accuracy: 0.6109 - val_loss: 1.3923 - val_auroc: 0.8918 - val_auprc: 0.6019 - val_accuracy: 0.5550
Epoch 2/100
296/296 [==============================] - ETA: 0s - loss: 0.7731 - auroc: 0.9633 - auprc: 0.8389 - accuracy: 0.7577
Epoch 00002: val_loss did not improve from 1.39231
296/296 [==============================] - 47s 157ms/step - loss: 0.7731 - auroc: 0.9633 - auprc: 0.8389 - accuracy: 0.7577 - val_loss: 2.7971 - val_auroc: 0.8034 - val_auprc: 0.4614 - val_accuracy: 0.4638
Epoch 3/100
296/296 [==============================] - ETA: 0s - loss: 0.6722 - auroc: 0.9721 - auprc: 0.870

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



296/296 [==============================] - ETA: 0s - loss: 0.0416 - auroc: 0.9997 - auprc: 0.9984 - accuracy: 0.9865
Epoch 00039: val_loss did not improve from 0.86846

Epoch 00039: ReduceLROnPlateau reducing learning rate to 0.000735091819660738.
296/296 [==============================] - 37s 125ms/step - loss: 0.0416 - auroc: 0.9997 - auprc: 0.9984 - accuracy: 0.9865 - val_loss: 1.4299 - val_auroc: 0.9275 - val_auprc: 0.7852 - val_accuracy: 0.7452
Epoch 40/100
296/296 [==============================] - ETA: 0s - loss: 0.0421 - auroc: 0.9998 - auprc: 0.9989 - accuracy: 0.9854
Epoch 00040: val_loss did not improve from 0.86846
296/296 [==============================] - 37s 126ms/step - loss: 0.0421 - auroc: 0.9998 - auprc: 0.9989 - accuracy: 0.9854 - val_loss: 0.9447 - val_auroc: 0.9534 - val_auprc: 0.8652 - val_accuracy: 0.8043
Epoch 41/100
296/296 [==============================] - ETA: 0s - loss: 0.0556 - auroc: 0.9992 - auprc: 0.9969 - accuracy: 0.9829
Epoch 00041: val_loss did not

In [17]:
# Evaluate
evaluator_model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=METRICS,
)
evaluator_model.evaluate(ds_test)

62/62 [==============================] - 4s 68ms/step - loss: 0.9058 - auroc: 0.9495 - auprc: 0.8493 - accuracy: 0.7999


[0.9057774543762207, 0.9494587779045105, 0.849315881729126, 0.7998641133308411]

In [18]:
evaluator_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 10)                20490     
Total params: 23,608,202
Trainable params: 0
Non-trainable params: 23,608,202
_________________________________________________________________
